So to find pattern in chess board, we can use the function, cv.findChessboardCorners(). We also need to pass what kind of pattern we are looking for, like 8x8 grid, 5x5 grid etc. 
Once we find the corners, we can increase their accuracy using cv.cornerSubPix(). We can also draw the pattern using cv.drawChessboardCorners().

In [ ]:
import numpy as np
import cv2 as cv
import glob #glob 的主要作用是帮助您根据文件名的模式或通配符快速筛选出所需的文件列表，以便进一步处理或操作这些文件。

#termination criteria 
#在最多进行30次迭代的情况下，如果迭代误差小于0.001，就停止迭代。这有助于确保标定过程在合理的时间内完成，并且达到所需的标定精度。
criteria = (cv.TermCriteria_EPS + cv.TERM_CRITERIA_MAX_ITER,30,0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp=  np.zeros((6*7,3),np.float32)
#将上述生成的二维坐标数组分配给 objp 的前两列。objp 是用于存储标定板上的物体点的数组。
# 前两列表示物体点的X和Y坐标，因此这行代码将X和Y坐标赋值给 objp 数组，而Z坐标则始终为0，因为这是一个平面标定板。
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2) #表示将 objp 数组的前两列（X和Y坐标）赋值为刚刚生成的标定板角点的二维坐标。

#Arrays to store objuect points and image points from all the image.
objpoints = [] # 3d point in real world space 
imgpoints = [] # 2d point in image plane

images = glob.glob('calibrationPic/*.jpg')

for fname in images:
    img  = cv.imread(fname)
    #将读取的图像 img 转换为灰度图像。这是因为相机标定通常在灰度图像上执行，以简化处理和提高精度。
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY) 
    
    # find the chess board corners
    # 在灰度图像中查找棋盘格的角点。参数 (7, 6) 指定了标定板的大小，即7行6列。ret 是一个标志，表示是否找到了角点。如果找到了角点，corners 变量将包含角点的坐标。
    ret,corners = cv.findChessboardCorners(gray,(7,6),None)
    
    #if found ,add object points ,image poins(after refining them)
    # 如果找到了，将物体点坐标 objp 添加到 objpoints 列表中。objp 包含了标定板上的物体点坐标，它是一个事先定义好的数组。
    if ret == True:
        objpoints.append(objp)
    '''
    执行了角点的亚像素级别精确化，并将精确化后的角点坐标添加到 imgpoints 列表中。下面是对这两行代码的逐行分析：
        corners2：这是一个变量，用于存储经过亚像素级别精确化后的角点坐标。

    cv.cornerSubPix()：这是OpenCV的函数，用于执行角点的亚像素级别精确化。它通过拟合像素值的亮度分布来提高角点坐标的精度。

    gray：这是灰度图像，通常是从彩色图像中转换而来的。亚像素级别的精确化通常在灰度图像上执行。

    corners：这是通过之前的函数（例如 cv.findChessboardCorners()）检测到的初始角点坐标。

    (11, 11)：这是搜索窗口的大小。cv.cornerSubPix() 将在一个局部窗口内搜索角点的亚像素级别位置。这个参数指定了窗口的大小。通常，较大的窗口可以提供更稳定的结果，但会增加计算成本。

    (-1, -1)：这是搜索的停止条件。在这种情况下，它表示搜索会持续直到达到最大迭代次数或指定的精度阈值，具体的停止条件由之前定义的 criteria 变量决定。
    '''    
    corners2 = cv.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
    imgpoints.append(corners2)
    
    # Draw and Display the corners
    cv.drawChessboardCorners(img,(7,6),corners2,ret)
    cv.imshow('img',img)
    cv.waitKey(500)
    
cv.destroyAllWindows()
    


## 1.Hiki calibration


In [14]:
import numpy as np
import cv2 as cv
import glob
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 50, 0.0001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((8*11,3), np.float32)
objp[:,:2] = np.mgrid[0:11,0:8].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('cali/ft006/*.bmp')
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (11,8), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
    corners2 = cv.cornerSubPix(gray,corners, (5,5), (-1,-1), criteria)
    imgpoints.append(corners2)
    # Draw and display the corners
    cv.drawChessboardCorners(img, (11,8), corners2, ret)
    cv.imshow('img', img)
    cv.waitKey(50)
cv.destroyAllWindows()

## 2.calibration 标定


Now that we have our object points and image points, we are ready to go for calibration. We can use the function, cv.calibrateCamera() which returns the camera matrix, distortion coefficients, rotation and translation vectors etc. 

根据检测到的物体点坐标 (objpoints) 和图像点坐标 (imgpoints) 来估计相机的内部参数和外部参数。

    ret：这是一个标志，用于表示相机标定是否成功。如果相机标定成功，ret 通常会被设置为 True 或一个非零值。

    mtx：这是相机的内部参数矩阵，通常包括焦距和主要点的信息。它是相机标定的一个重要输出。

    dist：这是相机的畸变参数，通常包括径向和切向畸变系数。它也是相机标定的一个重要输出。

    rvecs：这是旋转向量的数组，表示每幅图像的外部旋转参数。对于每个图像，它包含一个旋转向量，用于描述相机坐标系到图像坐标系的旋转。

    tvecs：这是平移向量的数组，表示每幅图像的外部平移参数。对于每个图像，它包含一个平移向量，用于描述相机坐标系到图像坐标系的平移。

    cv.calibrateCamera()：这是OpenCV的函数，用于执行相机标定。它将物体点坐标 (objpoints) 和图像点坐标 (imgpoints) 作为输入，以及图像的形状 (gray.shape[::-1])、畸变参数 (None) 和内部参数矩阵 (None) 作为可选参数。

    objpoints：这是物体点坐标的列表，包含每个图像的标定板上的物体点坐标。这些点的坐标是在物体坐标系中定义的。

    imgpoints：这是图像点坐标的列表，包含每个图像中检测到的标定板角点的坐标。这些点的坐标是在图像坐标系中定义的。

    gray.shape[::-1]：这是图像的形状，以元组的形式表示 (宽度, 高度)。由于 OpenCV 中常用的图像坐标系的表示方式是 (列, 行)，所以 gray.shape[::-1] 实际上是将图像的高度和宽度交换，以适应函数的参数要求。

总的来说，这行代码执行了相机标定操作，计算了相机的内部参数 (mtx)、畸变参数 (dist)、外部旋转参数 (rvecs) 和外部平移参数 (tvecs)，并返回一个标志 ret，表示标定是否成功。成功的相机标定对于后续的图像处理和计算相机姿态非常重要。

In [15]:
ret,mtx,dist,rvecs,tvecs = cv.calibrateCamera(objpoints,imgpoints,gray.shape[::-1],None,None)
print('instrinsic parameters')
print(mtx)
print('distortion coefficients')
print(dist)

instrinsic parameters
[[4.73943455e+03 0.00000000e+00 1.24485724e+03]
 [0.00000000e+00 4.73904291e+03 1.11336910e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
distortion coefficients
[[-3.18682291e-02  1.77413342e-01 -1.38907170e-04  5.77209995e-04
  -6.63996916e-01]]


In [16]:
fx = mtx[0][0]
fy = mtx[1][1]
cx = mtx[0][2]
cy = mtx[1][2]
print(fx,fy,cx,cy)

4739.434549652898 4739.042907576969 1244.857243975468 1113.3691041885147


保存内参到json文件

In [17]:
import json

# 要保存的Python数据
data = {
    "fx": fx,
    "fy": fy,
    "cx": cx,
    "cy": cy
}

# 指定要保存的JSON文件路径
json_file_path = "ftCalibrationData/ft006.json"

# 使用json.dump()将数据写入JSON文件
with open(json_file_path, "w") as json_file:
    json.dump(data, json_file)

print(f"数据已保存到 {json_file_path}")

数据已保存到 ftCalibrationData/ft006.json


In [10]:
# 打开已保存的JSON文件并加载数据
with open(json_file_path, "r") as json_file:
    loaded_data = json.load(json_file)

# 打印加载的数据
print("加载的数据：", loaded_data)
fx = loaded_data['fx']
print(fx)

加载的数据： {'fx': 4730.231591097344, 'fy': 4730.249962519248, 'cx': 1234.6417286366045, 'cy': 1074.1968971162278}
4730.231591097344


## Undistortion 去畸变
现在，我们可以取一张图像并进行去畸变处理。OpenCV提供了两种方法来实现这一点。然而首先，我们可以根据一个自由缩放参数来优化相机矩阵，使用cv.getOptimalNewCameraMatrix()方法。如果缩放参数alpha=0，它将返回带有最少不需要的像素的去畸变图像。因此，它甚至可能会移除图像角落的一些像素。如果alpha=1，所有像素都会被保留，并伴有一些额外的黑色图像。这个函数还会返回一个图像ROI，可用于裁剪结果。

In [5]:
img = cv.imread('calibrationPic/left12.jpg')
h,w = img.shape[:2]
'''
    mtx：这是相机的内部参数矩阵，之前通过相机标定计算得到的。

    dist：这是相机的畸变参数，也是通过相机标定计算得到的。

    (w, h)：这是图像的宽度和高度，用于指定图像的尺寸。

    1：这是一个缩放因子，通常设置为1，以便保持图像的原始尺寸。

    (w, h)：这是输出图像的尺寸，它与输入图像的尺寸相同。

cv.getOptimalNewCameraMatrix() 函数用于获取一个新的相机矩阵 newcameramtx，该矩阵可以用于矫正图像中的畸变。此外，还返回了感兴趣区域 (ROI) 的信息 roi，ROI 是一个矩形区域，其中包含了矫正后的图像。

使用这个新的相机矩阵和 ROI，您可以进行畸变矫正操作，以纠正图像中的畸变效应，使图像更准确地表示实际场景。这在计算机视觉和摄影领域中非常常见，以提高图像处理的准确性。
'''
newcameramtx,roi = cv.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

1. Using cv.undistort()
This is the easiest way. Just call the function and use ROI obtained above to crop the result. 

In [6]:
# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresulthiki.png', dst)

True

2. Using remapping

This way is a little bit more difficult. First, find a mapping function from the distorted image to the undistorted image. Then use the remap function. 

In [7]:
# undistort
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult2.png', dst)

True


Re-projection Error 
重投影误差可以很好地估计所找到的参数的精确程度。重投影误差越接近零，我们发现的参数就越准确。给定固有矩阵、畸变矩阵、旋转和平移矩阵，我们必须首先使用 cv.projectPoints() 将对象点转换为图像点。然后，我们可以计算通过变换得到的结果与角点查找算法之间的绝对范数。为了找到平均误差，我们计算所有校准图像计算的误差的算术平均值。

In [8]:
mean_error = 0
for i in range(len(objpoints)):
 imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
 error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
 mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )

total error: 0.006484001341287346


在上一次相机校准过程中，您已经找到了相机矩阵、畸变系数等。给定一个图案图像，我们可以利用上述信息来计算它的姿态，或者物体在空间中的位置，比如它是如何旋转的，对于平面物体，我们可以假设 Z=0，这样，现在的问题就变成了如何将相机放置在空间中以查看我们的图案图像。所以，如果我们知道物体在空间中的位置，我们就可以在其中画一些2D图来模拟3D效果。让我们看看如何做。我们的问题是，我们想要在棋盘的第一个角上绘制 3D 坐标轴（X、Y、Z 轴）。 X 轴为蓝色，Y 轴为绿色，Z 轴为红色。所以实际上，Z 轴应该感觉垂直于我们的棋盘平面。首先，我们从之前的校准结果中加载相机矩阵和畸变系数。

In [40]:
import numpy as np
import cv2 as cv
import glob

# 将数据保存到NPZ文件中
np.savez('camera_calibration.npz', mtx=mtx, dist=dist, rvecs=rvecs, tvecs=tvecs)


#load previously saved data
with np.load('camera_calibration.npz') as X:
    mtx,dist,_,_ = [X[i] for i in ('mtx','dist','rvecs','tvecs')]
    

创建一个名为 "draw" 的函数，该函数接受棋盘上的角点（使用 cv.findChessboardCorners() 获取）和用于绘制3D坐标轴的点。

In [41]:
def draw(img,corners,imgpts):
    # imgpts = np.int32(imgpts).reshape(-1,2)
    
    corner = tuple(corners[0].ravel())
    img = cv.line(img,corner,tuple(imgpts[0].ravel()),(255,0,0),5)
    img = cv.line(img,corner,tuple(imgpts[1].ravel()),(0,255,0),5)
    img = cv.line(img,corner,tuple(imgpts[2].ravel()),(0,0,255),5)
    return img

def drawCube(img,corners,imgpts):
    imgpts = np.int32(imgpts).reshape(-1,2)
    # draw ground floor in green
    img = cv.drawContours(img, [imgpts[:4]],-1,(0,255,0),-3)
    # draw pillars in blue color
    for i,j in zip(range(4),range(4,8)):
        img = cv.line(img, tuple(imgpts[i]), tuple(imgpts[j]),(255),3)
    # draw top layer in red color
    img = cv.drawContours(img, [imgpts[4:]],-1,(0,0,255),3)
    return img

然后，与之前的情况一样，我们创建终止条件、对象点（棋盘上角点的3D点）和坐标轴点。坐标轴点是用于绘制坐标轴的3D空间中的点。我们绘制长度为3的坐标轴（单位将以棋盘格大小为基准）。因此，我们的X轴从(0,0,0)绘制到(3,0,0)，Y轴也是如此。对于Z轴，它从(0,0,0)绘制到(0,0,-3)。负值表示它朝向相机方向绘制。

In [42]:
criteria = (cv.TermCriteria_EPS + cv.TERM_CRITERIA_MAX_ITER,50,0.0001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((8*11,3), np.float32)
objp[:,:2] = np.mgrid[0:11,0:8].T.reshape(-1,2)

# axis = np.float32([[3,0,0],[0,3,0],[0,0,-3]]).reshape(-1,3)
axis = np.float32([[0,0,0], [0,3,0], [3,3,0], [3,0,0],
 [0,0,-3],[0,3,-3],[3,3,-3],[3,0,-3] ])

现在，像往常一样，我们加载每张图像。搜索7x6的棋盘格。如果找到了，我们会使用子角点像素对其进行精化。然后，为了计算旋转和平移，我们使用函数cv.solvePnPRansac()。一旦我们得到了这些变换矩阵，我们就可以使用它们将我们的坐标轴点投影到图像平面上。简单来说，我们找到了图像平面上与3D空间中的每个点(3,0,0)、(0,3,0)、(0,0,3)相对应的点。一旦我们得到了它们，我们就使用我们的generateImage()函数从第一个角绘制到这些点的线。完成！

In [43]:
for fname in glob.glob('cali/*.bmp'):
    img = cv.imread(fname)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    ret,corners = cv.findChessboardCorners(gray,(11,8),None)
    if ret == True:
        corners2 = cv.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
    # find the rotation and translation vectors
    ret,rvecs,tvecs = cv.solvePnP(objp,corners,mtx,dist)
    
    #project 3D points to image plane
    imgpts,jac = cv.projectPoints(axis,rvecs,tvecs,mtx,dist)
    
    # img = draw(img,corners2,imgpts)
    img = drawCube(img,corners2,imgpts)
    
    cv.imshow('img',img)
    k = cv.waitKey(0) & 0xFF
    if k == ord('s'):
        cv.imwrite(fname[:15]+'.bmp',img)
cv.destroyAllWindows()